<a href="https://colab.research.google.com/github/Applied-Machine-Learning-2022/final-project-yeg-ua/blob/main/Yolov5_DeepSort_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [3]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet.git  # clone repo
%pip install -qr /content/requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.12.0+cu113 (Tesla T4)


# Download data

Get test video from repo and extract the first 2 seconds of it 

In [19]:
%cd /content/Yolov5_StrongSORT_OSNet

# get yolov5m model trained on the crowd-human dataset
!wget -nc https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet/releases/download/v.2.0/crowdhuman_yolov5m.pt -O /content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt

# get the test video from the repo
!wget -nc https://github.com/Applied-Machine-Learning-2022/final-project-yeg-ua/raw/main/palace.mp4

# copy video into another file
!ffmpeg  -i palace.mp4 -c copy out.mp4

/content/Yolov5_StrongSORT_OSNet
File ‘/content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt’ already there; not retrieving.
--2022-07-30 02:29:28--  https://github.com/Applied-Machine-Learning-2022/final-project-yeg-ua/raw/main/palace.mp4
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Applied-Machine-Learning-2022/final-project-yeg-ua/main/palace.mp4 [following]
--2022-07-30 02:29:28--  https://raw.githubusercontent.com/Applied-Machine-Learning-2022/final-project-yeg-ua/main/palace.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5909113 (5.6M) [application/octet-stream]
Saving 

## Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935. 

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time

In [20]:
!python track.py --yolo-weights /content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt --classes 0 --strong-sort-weights osnet_x0_25_msmt17.pt --source out.mp4 --save-vid

strong_sort/deep/reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
track: yolo_weights=['/content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt'], strong_sort_weights=osnet_x0_25_msmt17.pt, config_strongsort=strong_sort/configs/strong_sort.yaml, source=out.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=False, save_txt=False, save_conf=False, save_crop=False, save_vid=True, nosave=False, classes=[0], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/track, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False
YOLOv5 🚀 2022-7-30 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 308 layers, 21041679 parameters, 0 gradients
Downloading...
From: https://driv

# Show results

In [21]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
show_video('/content/Yolov5_StrongSORT_OSNet/runs/track/exp3/out.mp4')